In [3]:
import selenium
from selenium import webdriver
import time
import subprocess
import math
import os
import random
import tempfile


In [6]:
class Film(object):
    def __init__(self, height, width, filename):
        self.height = height
        self.width = width
        self.driver = webdriver.Firefox()
        self.driver.set_window_size(width, height)
        self.tmpdir = tempfile.mkdtemp()
        self.filename = filename
        self.sequence_position = 0
    
    def get_view_height(self):
        return int(self.driver.execute_script("return window.innerHeight"))
    
    def render_frames(self, top, bottom):
        for e,h in enumerate(range(top,bottom,1),start = self.sequence_position):
            self.driver.execute_script(f"window.scrollTo(0,{h});")
            dst = f"{self.tmpdir}/{str(e).zfill(5)}.png"
            self.driver.save_screenshot(dst)
            self.sequence_position = e+1
    
    def single(self, url, selector):
        self.driver.get(url)
        el = self.driver.find_element_by_css_selector(selector)
        
        top = el.location['y']
        bottom = int(el.location["y"]+el.size['height']-self.get_view_height())
        self.render_frames(top,bottom)
        
    def linear(self, url, selector_from, selector_to ):
        self.driver.get(url)
        el_top = self.driver.find_element_by_css_selector(selector_from)
        el_bottom = self.driver.find_element_by_css_selector(selector_to)
        
        top = el_top.location['y']
        bottom = int(el_bottom.location["y"]+el_bottom.size['height']-self.get_view_height())
        
        self.render_frames(top,bottom)
            
    def render(self):
        cmd = f"""ffmpeg -r 60 -i {self.tmpdir}/%05d.png -c:v libx264 -r 60 -pix_fmt yuv420p {self.filename} """
        subprocess.check_call(cmd, shell=True)

In [8]:
shot = {
    'height': 720,
    'width': 1280,
    'script': [
        {'type': 'single','params':{'selector':'.nyh_article','url':"https://www.svt.se/nyheter/inrikes/fyra-atalas-i-allra-harvan",}},
        {'type': 'linear','params':{'selector_from':'#main-content','selector_to':"#comments",'url':"https://svenska.yle.fi/artikel/2018/03/30/din-generation-hur-har-det-gatt-for-er-som-ar-fodda-samma-ar",}},
        
    ]
}
film = Film(shot["height"], shot["width"], "test_class.mp4")
for s in shot["script"]:
    fn = getattr(film, s["type"])
    fn(**s["params"])
film.render()